In [2]:
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import word_tokenize
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


[nltk_data] Downloading package punkt to C:\Users\Nischay
[nltk_data]     kapoor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
text = """
 Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: ‘I don’t really hide any feelings too much.
I think everyone knows this is my job here. When I’m on the courts or when I’m on the court playing, I’m a competitor and I want to beat every single person whether they’re in the locker room or across the net.
So I’m not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
I’m a pretty competitive girl. I say my hellos, but I’m not sending any players flowers as well. Uhm, I’m not really friendly or close to many players.
I have not a lot of friends away from the courts.’ When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men’s tour than the women’s tour? ‘No, not at all.
I think just because you’re in the same sport doesn’t mean that you have to be friends with everyone just because you’re categorized, you’re a tennis player, so you’re going to get along with tennis players.
I think every person has different interests. I have friends that have completely different jobs and interests, and I’ve met them in very different parts of my life.
I think everyone just thinks because we’re tennis players we should be the greatest of friends. But ultimately tennis is just a very small part of what we do.
There are so many other things that we’re interested in, that we do.
"""
tokens = word_tokenize(text.lower())
tokens = [word for word in tokens if word.isalpha()]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)
sequences = []

for i in range(3, len(tokens)):
    seq = tokens[i-3:i+1] 
    sequences.append(seq)

sequences = tokenizer.texts_to_sequences(sequences)
sequences = np.array(sequences)

X, y = sequences[:, :-1], sequences[:, -1]
vocab_size = len(tokenizer.word_index) + 1
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

In [4]:
# GRU Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

gru_model = Sequential()
gru_model.add(Embedding(vocab_size, 10, input_length=3))
gru_model.add(GRU(32)) 
gru_model.add(Dense(vocab_size, activation='softmax'))

gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0102 - loss: 4.9758  
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0515 - loss: 4.9670
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0619 - loss: 4.9587
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0528 - loss: 4.9462
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0631 - loss: 4.9289
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0630 - loss: 4.8942
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0557 - loss: 4.8457
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0490 - loss: 4.7762
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0398 - loss: 4.7126   
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0697 - loss: 4.5560 
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0469 - loss: 4.5356     
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 

In [5]:
def predict_next_word(model, tokenizer, seed_text):
    tokens = word_tokenize(seed_text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    encoded = tokenizer.texts_to_sequences([tokens])[-1]
    padded = pad_sequences([encoded], maxlen=3, truncating='pre')
    pred = model.predict(padded, verbose=0)
    next_word = tokenizer.index_word[np.argmax(pred)]
    return next_word

In [6]:
print("GRU prediction:", predict_next_word(gru_model, tokenizer, "tennis players on"))


GRU prediction: the
